## Integration tests of federated data in time series 

In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append(".../S-ADL")


In [ ]:
from flexanomalies.datasets.preprocessing_utils import scaling
from SADL.federated_data.algorithms import flexanomalies
from SADL.time_series.time_series_utils import TimeSeriesProcessor
from SADL.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split

##### Example of time series data set for testing and preprocessiong

In [ ]:
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"]
X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding

In [ ]:
# from SADL.static_data.preprocessing.preprocessing_static import OneHotEncoderPreprocessing
# encoder = OneHotEncoderPreprocessing(columns=['Type'])
# data_encoded = encoder.fit_transform(X)

In [9]:
from SADL.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing 
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

#### Autoencoder Model for anomaly detection in time series 

In [ ]:
kwargs = {
    "algorithm_": "autoencoder",
    "contamination":0.1,
    "label_parser": None,
    "epochs": 3,
    "input_dim": X.shape[1],
    "batch_size": 16,
    "neurons": [16, 8, 16],
    "hidden_act": ["relu", "relu", "relu"],
    "preprocess":False,
    "w_size": 10,
    "n_pred": 1,
     "n_clients":3,
     "n_rounds":5,
     }



modelAE = flexanomalies.FlexAnomalyDetection(**kwargs)
print(modelAE.get_params())


In [ ]:
processor = TimeSeriesProcessor(window_size= kwargs["w_size"], step_size=1, future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

In [ ]:
modelAE.fit(X_train_windows,y_train_windows)


In [ ]:
scores_pred = modelAE.decision_function(X_train_windows)
print("Scores",scores_pred)
modelAE.predict(X_train_windows)

In [ ]:
modelAE.model.d_scores_ ==scores_pred
modelAE.model.labels_


In [ ]:
modelAE.evaluate(X_test_windows, X_test_windows, label_test = y_test_windows.flatten().astype("int"))

#### CNN_LSTM model for anomaly forcasting  in time series 

In [ ]:
kwargs = {
    "algorithm_": "deepCNN_LSTM",
    "contamination":0.1,
    "label_parser": None,
    "epochs": 5,
    "input_dim": X.shape[1],
    "batch_size": 8,
    "filters_cnn": [8, 6],
    "units_lstm": [8,6],
    "kernel_size": [4,4],
    "hidden_act": ["relu", "relu"],
    "w_size": 20,
    "n_pred": 1,
     "n_clients":2,
     "n_rounds":3,
     }



modelDeep = flexanomalies.FlexAnomalyDetection(**kwargs)


In [ ]:
processor =   TimeSeriesProcessor(window_size= kwargs["w_size"], step_size=1, future_prediction=True, n_pred=kwargs["n_pred"])
X_train_windows, y_train_windows, X_test_windows, y_test_windows, l_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test,l_test=y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)
print("l_test shape:",l_test_windows.shape)

In [ ]:
modelDeep.fit(X_train_windows,y_train_windows)


In [ ]:
modelDeep.predict(X_train)



In [ ]:
modelDeep.evaluate(X_test_windows, y_test_windows, label_test = l_test_windows.flatten().astype("int"))